In [ ]:
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator
# Adjust the path to the directory containing 'scripts/algo'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts')))
from algo.qft_qpe_qdrift import *
from utils.scalable_numerical_tests import *